#### Load required Liabraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import shap
from scorecardutils.feature_selection import (shap_feature_selection,
                                              find_correlation_groups,
                                              select_best_features_from_corr_groups,
                                              vsi_check)
from scorecardutils.BivariatePlot import enhanced_bivariate_plot,transform_and_plot_oot_bivariate_data

#from feature_engine.selection import SmartCorrelatedSelection


In [ ]:
dev_data = pd.read_csv('../data/credit_risk_dataset.csv')

In [ ]:
dev_data.shape

In [ ]:
dev_data.head()

In [ ]:
## Make sure each feature has correct data types --float,int,catgeory
for col in dev_data.select_dtypes(include='object').columns:
    dev_data[col] = dev_data[col].astype('category')

In [ ]:
## Define target and features names
target = 'default'
features = dev_data.drop(columns=[target]).columns.tolist()

In [ ]:
xgb_params = {
    "objective": "binary:logistic",       # For binary classification
    "eval_metric": "auc",                 # auc
    "learning_rate": 0.05,                 # Smaller means more trees; safer
    "max_depth": 6,                       # Controls model complexity
    "subsample": 0.8,                     # Fraction of samples per tree
    "colsample_bytree": 0.8,              # Fraction of features per tree
    "lambda": 1,                          # L2 regularization
    "alpha": 0,                           # L1 regularization
    "n_estimators": 100,                  # Total trees
}

In [ ]:
selected_features,importance_df,_ =shap_feature_selection(train_data=dev_data,feature_names=features,target_name='default',verbose=True,
                                                        test_size=0.3,random_state=42,use_train_for_shap=False,
                                                        model_params=xgb_params)

In [ ]:
_,importance_df,shapDF =shap_feature_selection(train_data=dev_data,feature_names=selected_features,target_name='default',verbose=False,
                                                        split_data=False,random_state=42,
                                                        model_params=xgb_params,create_shap_df=True)

In [ ]:
correlated_groups = find_correlation_groups(shapDF, corr_threshold=0.8)

In [ ]:
correlated_groups

In [ ]:
_,selected_features_corr = select_best_features_from_corr_groups(correlated_groups, feature_importance_df=importance_df,
                                      feature_importance_col='SHAP_Importance',
                                      feature_name_col='Feature')

In [ ]:
selected_features_corr

### Variable Binning and IV ,Stability Check

In [ ]:
from optbinning import BinningProcess
from sklearn.model_selection import train_test_split

In [ ]:
syndata= pd.read_csv('../data/synthetic_binary_classification_data.csv')

In [ ]:
target = 'target'

In [ ]:
X= syndata.drop(columns=[target])
y= syndata[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42,stratify=y)

In [ ]:
# Combine
train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)
train_data.head()

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
model_vars= X_train.columns.tolist()

In [ ]:
## Extract categorical and numerical columns
categorical_columns = X_train.select_dtypes(include=['object','category']).columns.tolist()
numerical_columns = X_train.select_dtypes(include=['number']).columns.tolist()

In [ ]:
categorical_columns

In [ ]:
numerical_columns

In [ ]:
X_train.head()

In [ ]:
"""
Define Any Special Codes to treat them in separate bucket
e.g 
special_codes = [-999,-1,-2]
or if need to treat separate codes with different values in different special buckets
special_codes = {'special_1': -9, "special_2": -8, "special_3": -7}
"""

special_codes = None

In [ ]:
"""
Define Dictionary with optimal binning fitting options for specific variables. we can update it as per our customization
for specific attributes.
split_digits: If split_digits is set to 0, the split points are integers otherwise the split points are rounded 
to the number of digits specified by split_digits.
user_splits: If user_splits is set, the splits are fixed to the values specified by user_splits.
user_splits_fixed: If user_splits_fixed is set to True, the splits are fixed to the values specified by user_splits.
monotonic_trend: If monotonic_trend is set to 'ascending', the bad rate should be non-decreasing.
cat_cutoff:Generate bin others with categories in which the fraction of occurrences is below the cat_cutoff value. 
i.e If cat_cutoff is set to 0.05, the bin will be generated with categories in which the fraction of occurrences is below the cat_cutoff value.

Initially can be passed as None. binning_fit_params=None

binning_fit_params = {
    "dti": {"monotonic_trend": "ascending","split_digits":2 ,
            "user_splits": [ 8.89, 10.91, 14.68, 16.03,18.23, 20.8 , 22.11, 28.37],
           # "user_splits_fixed" :[True,True] 
           }
    }



"""
binning_fit_params = {

    "loyalty_score":{"split_digits":2,
        "user_splits": [    -3.07959914, -2.46296906, -1.8946799 , -1.6182403 , -1 ,
                            0,  0.37816253,  0.77244589,  1.2557528 ,
                            1.77894938],
        "user_splits_fixed": [False,False,False,False,True,True,False,False,False,False]
    }
}




In [ ]:
"""
By default:optbinning assigns WoE = 0 and event rate = overall mean to special values (like missing or outliers), unless you override it.
Update Strategy for Special Values:
Strategy A: Neutralize Special Values i.e.Assign WoE = 0 (effectively no contribution to the score).
Strategy B: Assign Empirical WoE for Special Values i.e.Let the special values’ WoE be calculated based on their actual event rate in the data.This is often used when special codes have meaningful predictive power
Strategy C: Merge Special Value into Closest Bin. f a special code behaves like a particular bin (e.g., 999 behaves like bin [30–40]), assign its WoE manually to match that bin’s WoE

binning_transform_params ={
   'revol_util':{'metric_special':'empirical'},
    'dti':{'metric_special': -0.306345},
    'inq_last_6mths':{'metric_special':'empirical'}
}

or 
binning_transform_params = None
"""
binning_transform_params = {
    'age':{'metric_missing':'empirical','metric_special':'empirical'}
}

In [ ]:
"""
List of variables to be fixed. 
The binning process will retain these variables if the selection criteria is not satisfied.
"""
fixed_variables=None

In [ ]:
"""
Define the selection criteria for the binning process
selection_criteria = {
    "iv": {"min": 0.01, "max": 0.5,"strategy": "highest", "top": 50},
    "quality_score": {"min": 0.01}
}

or 
selection_criteria = None
"""
selection_criteria = {
    "iv": {"min": 0.01,"strategy": "highest", "top": 50}
}

In [ ]:
binning_process = BinningProcess(variable_names=model_vars, special_codes=special_codes,
                                 categorical_variables=categorical_columns,
                                 selection_criteria=selection_criteria,
                                 binning_fit_params=binning_fit_params,
                                 binning_transform_params=binning_transform_params,
                                fixed_variables=fixed_variables)

In [ ]:
import warnings
warnings.filterwarnings("ignore")


In [ ]:
# Fit the binning process    
binning_process.fit(X=X_train[model_vars], y=y_train.values)

In [ ]:
# We can save all the variables passed to binning process and their IVs for manual screening
iv_tab=binning_process.summary().sort_values(by='iv',ascending=False)
#iv_tab.to_excel('iv_tab.xlsx', index=False)
iv_tab

In [ ]:
iv_selected_variables = iv_tab[iv_tab['iv']>0.02]['name'].tolist()

In [ ]:
iv_selected_variables

In [ ]:
## To see any specific variable binning table
optb = binning_process.get_binned_variable('customer_group')
df = optb.binning_table.build()
df

In [ ]:
### CSI summary
csi_summ = vsi_check(
    X_oot=X_test, 
    X_train=X_train,
    binning_process=binning_process,
    style='summary',  # or 'detailed' for bin-level information
    psi_min_bin_size=0.01,
    max_workers=4  # Adjust based on your CPU cores
)

## CSI detailed Summary
csi_det = vsi_check(
    X_oot=X_test, 
    X_train=X_train,
    binning_process=binning_process,
    style='detailed',  # or 'detailed' for bin-level information
    psi_min_bin_size=0.01,
    max_workers=4  # Adjust based on your CPU cores
)

In [ ]:
# to save csi in excel
with pd.ExcelWriter('csi.xlsx', engine='openpyxl', mode='w') as writer:
    csi_summ.to_excel(writer, sheet_name='summary', index=False)
    csi_det.to_excel(writer, sheet_name='detail', index=False)

In [ ]:
# Filter stable variables (PSI < threshold)
stable_variables = [str(var) for var in csi_summ[csi_summ['CSI'] < 10]['Variable'].tolist()]
    
# Filter unstable variables (PSI >= threshold)
unstable_variables = [str(var) for var in csi_summ[csi_summ['CSI'] >= 10]['Variable'].tolist()]
print("Unstable Variables:", unstable_variables)

In [ ]:
selected_features_ivs = [feature for feature in iv_selected_variables if feature not in unstable_variables]

### Get Bivariate plot on both Train/Test

In [ ]:
optb = binning_process.get_binned_variable('age')

# Get DataFrame from binning table
df = optb.binning_table.build()
df = df.drop(columns='JS')
df = df.reset_index()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np
from openpyxl.drawing.image import Image
from openpyxl import Workbook
from typing import List, Optional, Union, Dict, Any, Tuple

def unified_bivariate_analysis(
    binning_process,
    filename: str = 'bivariate_analysis',
    metric: str = 'event_rate',
    variables: Optional[List[str]] = None,
    figsize: tuple = (12, 6),
    dpi: int = 100,
    style: str = 'whitegrid',
    oot_data: Optional[pd.DataFrame] = None,
    target_column: Optional[str] = None,
    compare_data: bool = True,
    show_bar_values: bool = False,
    verbose: bool = False
) -> None:
    """
    Unified function to create bivariate analysis plots for binned data with options for:
    1. Training data only
    2. OOT data only (based on training binning)
    3. Comparison between training and OOT data
    
    Parameters:
    -----------
    binning_process : OptimalBinning process object
        The binning process containing the variables to plot
    filename : str, default='bivariate_analysis'
        Base name for the output Excel file
    metric : str, default='event_rate'
        Metric to plot. Options: 'event_rate', 'woe'
    variables : list of str, optional
        List of specific variable names to process. If None, all variables are processed.
    figsize : tuple, default=(12, 6)
        Figure size for plots in inches (width, height)
    dpi : int, default=100
        Resolution of saved images
    style : str, default='whitegrid'
        Seaborn style for plots
    oot_data : pandas DataFrame, optional
        The OOT dataset to transform and analyze. If None, only training data is plotted.
    target_column : str, optional
        Name of the target/outcome column in the OOT data. Required if oot_data is provided.
    compare_data : bool, default=True
        Whether to plot training data metrics alongside OOT data metrics.
        Only relevant when oot_data is provided.
    show_bar_values : bool, default=False
        Whether to show Count (%) values on top of the bars
    verbose : bool, default=False
        Whether to print detailed information during processing
    """
    # Validate inputs
    if oot_data is not None and target_column is None:
        raise ValueError("target_column must be specified when oot_data is provided")
    
    # Set mode based on inputs
    if oot_data is None:
        mode = "train_only"
    else:
        mode = "oot_comparison" if compare_data else "oot_only"
        
    # Validate metric parameter
    metric = metric.lower()
    if metric not in ['event_rate', 'woe']:
        raise ValueError("metric must be either 'event_rate' or 'woe'")
    
    # Get all available variables from binning process
    try:
        # Try to get variable names directly
        all_variables = binning_process.variable_names
    except AttributeError:
        # Fall back to get_support if variable_names is not available
        try:
            all_variables = binning_process.get_support(names=True)
        except AttributeError:
            raise ValueError("Unable to extract variable names from binning_process")
    
    # Filter variables if specified
    if variables is not None:
        # Check if all specified variables exist in the binning process
        invalid_vars = set(variables) - set(all_variables)
        if invalid_vars:
            raise ValueError(f"Variables not found in binning process: {', '.join(invalid_vars)}")
        selected_vars = variables
    else:
        selected_vars = all_variables
    
    # Track if we've created at least one valid sheet for Excel
    valid_sheets_created = False
    
    # Set up the visualization style
    plt.style.use('default')  # Clean matplotlib style
    
    # Create a temporary directory for images if it doesn't exist
    if not os.path.exists('temp_plots'):
        os.makedirs('temp_plots')
    
    # Dictionary to store plot paths and binning tables
    plot_paths = {}
    oot_binning_tables = {}
    
    # Determine column names based on metric
    metric_column = 'Event rate' if metric == 'event_rate' else 'WoE'
    y_axis_label = 'Event Rate' if metric == 'event_rate' else 'Weight of Evidence (WoE)'
    
    # Create Excel writer
    with pd.ExcelWriter(f'{filename}.xlsx', engine='openpyxl', mode='w') as writer:
        # Process each selected variable
        for var in selected_vars:
            try:
                # Sanitize variable name for Excel sheet name (remove invalid characters)
                sheet_name = str(var)
                for char in ['[', ']', ':', '*', '?', '/', '\\']:
                    sheet_name = sheet_name.replace(char, '_')
                # Ensure sheet name is not longer than 31 characters (Excel limit)
                if len(sheet_name) > 31:
                    sheet_name = sheet_name[:31]
                    
                # Get the binned variable from the trained process
                try:
                    optb = binning_process.get_binned_variable(var)
                except Exception as e:
                    if verbose:
                        print(f"Error getting binned variable {var}: {str(e)}")
                    continue
                
                # Get original binning table from training data
                train_df = optb.binning_table.build()
                if 'JS' in train_df.columns:
                    train_df = train_df.drop(columns='JS')
                
                # Calculate IV if it doesn't exist
                if 'IV' not in train_df.columns:
                    # Calculate IV for each bin
                    train_df['IV'] = train_df.apply(
                        lambda row: (row['Non-event (%)'] - row['Event (%)']) * row['WoE'] 
                        if row.name != 'Totals' else np.NaN, 
                        axis=1
                    )
                    
                    # Calculate total IV (sum of all bin IVs)
                    total_iv = train_df['IV'].sum()
                    
                    # Add total IV to the Totals row if it exists
                    if 'Totals' in train_df.index:
                        train_df.loc['Totals', 'IV'] = total_iv
                
                # Keep the original DataFrame for Excel output including Totals row
                train_df_for_excel = train_df.reset_index()
                
                # Add Dataset column to training data
                train_df_for_excel.insert(0, 'Dataset', 'Training')
                
                # Extract the Totals row and prepare for plotting
                train_totals_row = None
                train_df_no_totals = train_df
                if 'Totals' in train_df.index:
                    train_totals_row = train_df.loc['Totals']
                    train_df_no_totals = train_df.drop('Totals')
                
                # Reset index for plotting
                train_df_reset = train_df_no_totals.reset_index()
                
                # The actual bin column is 'Bin', but if it doesn't exist, use the first column as fallback
                bin_col_name = 'Bin' if 'Bin' in train_df_reset.columns else train_df_reset.columns[0]
                
                # Process OOT data if provided
                oot_df = None
                oot_df_with_totals = None
                total_event_rate_oot = None
                total_iv_oot = None
                
                if oot_data is not None:
                    # Transform the OOT data for this variable using the trained binning
                    try:
                        X_var_oot = oot_data[var].values
                        y_oot = oot_data[target_column].values
                    except KeyError as e:
                        if verbose:
                            print(f"Variable {var} or target column {target_column} not found in OOT data: {str(e)}")
                        continue
                    
                    # Use the transform method with "bins" metric to get bin assignments
                    try:
                        oot_bins = optb.transform(X_var_oot, metric="bins")
                    except Exception as e:
                        if verbose:
                            print(f"Error transforming OOT data for variable {var}: {str(e)}")
                        continue
                    
                    # Create a dictionary to store counts for each bin - convert to strings to make hashable
                    bin_n_event = {str(bin_name): 0 for bin_name in train_df_reset[bin_col_name]}
                    bin_n_nonevent = {str(bin_name): 0 for bin_name in train_df_reset[bin_col_name]}
                    
                    # Count events and non-events for each bin in OOT data
                    for i, bin_label in enumerate(oot_bins):
                        bin_label_str = str(bin_label)
                        if bin_label_str in bin_n_event:
                            if y_oot[i] == 1:
                                bin_n_event[bin_label_str] += 1
                            else:
                                bin_n_nonevent[bin_label_str] += 1
                    
                    # Calculate totals for percentages
                    total_events_oot = sum(bin_n_event.values())
                    total_nonevents_oot = sum(bin_n_nonevent.values())
                    total_records_oot = total_events_oot + total_nonevents_oot
                    
                    # Create a new DataFrame for OOT data metrics
                    oot_stats = []
                    total_iv_oot = 0
                    for bin_name in train_df_reset[bin_col_name]:
                        bin_name_str = str(bin_name)
                        n_event = bin_n_event[bin_name_str]
                        n_nonevent = bin_n_nonevent[bin_name_str]
                        n_records = n_event + n_nonevent
                        
                        # Calculate event rate for this bin
                        event_rate = n_event / n_records if n_records > 0 else 0
                        
                        # Calculate event and non-event percentages
                        event_pct = n_event / total_events_oot if total_events_oot > 0 else 0
                        nonevent_pct = n_nonevent / total_nonevents_oot if total_nonevents_oot > 0 else 0
                        
                        # Calculate WoE similar to the binning_table.build() method
                        if n_event > 0 and n_nonevent > 0 and total_events_oot > 0 and total_nonevents_oot > 0:
                            p_event = n_event / total_events_oot
                            p_nonevent = n_nonevent / total_nonevents_oot
                            woe = np.log(p_nonevent / p_event)
                        else:
                            woe = 0  # Default value
                        
                        # Calculate IV for this bin
                        iv = (nonevent_pct - event_pct) * woe
                        total_iv_oot += iv
                        
                        oot_stats.append({
                            bin_col_name: bin_name,  # Keep as original type for consistency
                            'Count': n_records,
                            'Count (%)': n_records / total_records_oot if total_records_oot > 0 else 0,
                            'Non-event': n_nonevent,
                            'Non-event (%)': nonevent_pct,
                            'Event': n_event,
                            'Event (%)': event_pct,
                            'Event rate': event_rate,
                            'WoE': woe,
                            'IV': iv
                        })
                    
                    # Create DataFrame from collected statistics
                    oot_df = pd.DataFrame(oot_stats)
                    
                    # Calculate total event rate for OOT
                    total_event_rate_oot = total_events_oot / total_records_oot if total_records_oot > 0 else 0
                    
                    # Add totals row for OOT
                    totals_row_oot = {
                        bin_col_name: 'Totals',
                        'Count': total_records_oot,
                        'Count (%)': 1.0,
                        'Non-event': total_nonevents_oot,
                        'Non-event (%)': 1.0,
                        'Event': total_events_oot,
                        'Event (%)': 1.0,
                        'Event rate': total_event_rate_oot,
                        'WoE': '',  # WoE doesn't make sense for totals
                        'IV': total_iv_oot
                    }
                    
                    # Add totals row to OOT data
                    oot_df_with_totals = pd.concat([oot_df, pd.DataFrame([totals_row_oot])], ignore_index=True)
                    
                    # Add Dataset column to OOT data
                    oot_df_with_totals.insert(0, 'Dataset', 'OOT')
                    
                    # Store OOT binning table
                    oot_binning_tables[var] = oot_df_with_totals
                
                # Calculate missing Event (%) and Non-event (%) for Training data if they don't exist
                if 'Event (%)' not in train_df_for_excel.columns or train_df_for_excel['Event (%)'].isna().all():
                    # Check if 'Totals' row exists
                    totals_mask = train_df_for_excel[bin_col_name] == 'Totals'
                    
                    if totals_mask.any():
                        # Get total events from Totals row
                        total_events_train = train_df_for_excel.loc[totals_mask, 'Event'].values[0]
                        total_nonevents_train = train_df_for_excel.loc[totals_mask, 'Non-event'].values[0]
                        
                        # Calculate percentages only for non-Totals rows
                        non_totals_mask = ~totals_mask
                        train_df_for_excel.loc[non_totals_mask, 'Event (%)'] = train_df_for_excel.loc[non_totals_mask, 'Event'] / total_events_train
                        train_df_for_excel.loc[non_totals_mask, 'Non-event (%)'] = train_df_for_excel.loc[non_totals_mask, 'Non-event'] / total_nonevents_train
                        
                        # Set Totals row percentages to 1.0
                        train_df_for_excel.loc[totals_mask, 'Event (%)'] = 1.0
                        train_df_for_excel.loc[totals_mask, 'Non-event (%)'] = 1.0
                    else:
                        # Calculate totals directly from data (excluding totals row if it exists)
                        total_events_train = train_df_for_excel['Event'].sum()
                        total_nonevents_train = train_df_for_excel['Non-event'].sum()
                        
                        # Add a Totals row
                        totals_row = {
                            'Dataset': 'Training',
                            bin_col_name: 'Totals',
                            'Count': total_events_train + total_nonevents_train,
                            'Count (%)': 1.0,
                            'Non-event': total_nonevents_train,
                            'Non-event (%)': 1.0,
                            'Event': total_events_train,
                            'Event (%)': 1.0,
                            'Event rate': total_events_train / (total_events_train + total_nonevents_train) if (total_events_train + total_nonevents_train) > 0 else 0,
                            'WoE': '',
                            'IV': train_df_for_excel['IV'].sum() if 'IV' in train_df_for_excel.columns else 0
                        }
                        train_df_for_excel = pd.concat([train_df_for_excel, pd.DataFrame([totals_row])], ignore_index=True)
                        
                        # Calculate percentages for all non-Totals rows
                        train_df_for_excel.loc[train_df_for_excel[bin_col_name] != 'Totals', 'Event (%)'] = (
                            train_df_for_excel.loc[train_df_for_excel[bin_col_name] != 'Totals', 'Event'] / total_events_train
                        )
                        train_df_for_excel.loc[train_df_for_excel[bin_col_name] != 'Totals', 'Non-event (%)'] = (
                            train_df_for_excel.loc[train_df_for_excel[bin_col_name] != 'Totals', 'Non-event'] / total_nonevents_train
                        )
                
                # Check if we have double Totals rows in training data and remove duplicates if necessary
                totals_rows = train_df_for_excel[train_df_for_excel[bin_col_name] == 'Totals']
                if len(totals_rows) > 1:
                    # Keep only the first Totals row
                    train_df_for_excel = pd.concat([
                        train_df_for_excel[train_df_for_excel[bin_col_name] != 'Totals'],
                        totals_rows.iloc[[0]]
                    ]).reset_index(drop=True)
                
                # Determine which data to write to Excel based on mode
                if mode == "train_only":
                    # Add variable name header
                    var_header = pd.DataFrame([{col: '' for col in train_df_for_excel.columns}])
                    var_header['Dataset'] = f'{var}'
                    
                    # Combine header and data
                    df_for_excel = pd.concat([var_header, train_df_for_excel], ignore_index=True)
                    
                elif mode == "oot_only":
                    # Add variable name header
                    var_header = pd.DataFrame([{col: '' for col in oot_df_with_totals.columns}])
                    var_header['Dataset'] = f'{var}'
                    
                    # Combine header and data
                    df_for_excel = pd.concat([var_header, oot_df_with_totals], ignore_index=True)
                    
                else:  # oot_comparison - write both with headers and gap
                    # Ensure both dataframes have the same columns in the same order
                    all_columns = list(set(train_df_for_excel.columns) | set(oot_df_with_totals.columns))
                    
                    # Move 'Dataset' column to the beginning if present
                    if 'Dataset' in all_columns:
                        all_columns.remove('Dataset')
                        all_columns = ['Dataset'] + all_columns
                    
                    # Ensure both dataframes have all columns
                    for col in all_columns:
                        if col not in train_df_for_excel.columns:
                            train_df_for_excel[col] = ''
                        if col not in oot_df_with_totals.columns:
                            oot_df_with_totals[col] = ''
                    
                    # Reorder columns to match
                    train_df_for_excel = train_df_for_excel[all_columns]
                    oot_df_with_totals = oot_df_with_totals[all_columns]
                    
                    # Add variable name header
                    var_header = pd.DataFrame([{col: '' for col in all_columns}])
                    var_header['Dataset'] = f'{var}'
                    
                    # Add training data header
                    train_header = pd.DataFrame([{col: '' for col in all_columns}])
                    train_header['Dataset'] = 'Training Data'
                    
                    # Create a blank row for spacing
                    blank_row = pd.DataFrame([{col: '' for col in all_columns}])
                    
                    # Add OOT data header
                    oot_header = pd.DataFrame([{col: '' for col in all_columns}])
                    oot_header['Dataset'] = 'OOT Data'
                    
                    # Combine all parts: variable name, training header, training data, blank row, OOT header, OOT data
                    df_for_excel = pd.concat([
                        var_header,
                        train_header, 
                        train_df_for_excel, 
                        blank_row, 
                        oot_header, 
                        oot_df_with_totals
                    ], ignore_index=True)
                
                # Reorder columns to the desired structure
                desired_column_order = [
                    'Dataset', 
                    bin_col_name, 
                    'Count', 
                    'Count (%)', 
                    'Non-event', 
                    'Non-event (%)', 
                    'Event', 
                    'Event (%)', 
                    'Event rate', 
                    'WoE', 
                    'IV'
                ]
                
                # Add any missing columns that might be in df_for_excel but not in our desired order
                for col in df_for_excel.columns:
                    if col not in desired_column_order and col != 'index':
                        desired_column_order.append(col)
                
                # Reorder columns (only for columns that exist in the DataFrame)
                existing_columns = [col for col in desired_column_order if col in df_for_excel.columns]
                df_for_excel = df_for_excel[existing_columns]
                
                # Write to Excel
                df_for_excel.to_excel(writer, sheet_name=sheet_name, index=False)
                valid_sheets_created = True
                
                # Create plot image path
                plot_image_path = f'temp_plots/variable_{var}_{metric}_{mode}.png'
                
                # Create the appropriate plot based on mode
                fig, ax1 = plt.subplots(figsize=figsize)
                
                if mode == "train_only":
                    # Plot training data only
                    plot_single_dataset(
                        ax1, train_df_reset, bin_col_name, metric_column, 
                        y_axis_label, train_totals_row, "Training", line_color='darkgoldenrod', 
                        bar_color='#0a3f7d', show_bar_values=show_bar_values
                    )
                    plt.title(f'{var}: {y_axis_label} by Bin (Training Data)', fontsize=12)
                    
                elif mode == "oot_only":
                    # Plot OOT data only
                    plot_single_dataset(
                        ax1, oot_df, bin_col_name, metric_column, 
                        y_axis_label, pd.Series({'Event rate': total_event_rate_oot, 'WoE': '', 'IV': total_iv_oot}), 
                        "OOT", line_color='darkgoldenrod', bar_color='#0a3f7d', show_bar_values=show_bar_values
                    )
                    plt.title(f'{var}: {y_axis_label} by Bin (OOT Data)', fontsize=12)
                    
                else:  # oot_comparison
                    # Plot comparison between training and OOT data
                    plot_comparison(
                        ax1, train_df_reset, oot_df, bin_col_name, metric_column, 
                        y_axis_label, train_totals_row, total_event_rate_oot, metric, show_bar_values=show_bar_values
                    )
                    plt.title(f'{var}: {y_axis_label} Comparison (Training vs OOT)', fontsize=12)
                
                # Adjust layout and save
                plt.tight_layout()
                plt.savefig(plot_image_path, dpi=dpi, bbox_inches='tight')
                plt.close()
                
                # Store path for cleanup
                plot_paths[var] = plot_image_path
                
                # Insert plot image into Excel
                try:
                    img = Image(plot_image_path)
                    
                    # Start plot after the data table with some margin
                    row_position = len(df_for_excel) + 4
                    img.anchor = f'A{row_position}'
                    
                    writer.sheets[sheet_name].add_image(img)
                    
                    # Adjust column widths
                    for idx, col in enumerate(df_for_excel.columns):
                        column_width = max(len(str(col)), df_for_excel[col].astype(str).map(len).max())
                        writer.sheets[sheet_name].column_dimensions[chr(65 + idx)].width = column_width + 2
                except Exception as e:
                    if verbose:
                        print(f"Error adding image to Excel for variable {var}: {str(e)}")
            
            except Exception as e:
                if verbose:
                    print(f"Error processing variable {var}: {str(e)}")
                continue
        
        # Add a dummy sheet if no valid sheets were created to avoid Excel error
        if not valid_sheets_created:
            dummy_df = pd.DataFrame({'Message': ['No valid variables found for analysis']})
            dummy_df.to_excel(writer, sheet_name='Info', index=False)
            if verbose:
                print("No valid sheets were created. Adding a dummy sheet.")
    
    # Clean up temporary image files
    for path in plot_paths.values():
        if os.path.exists(path):
            os.remove(path)
    
    # Remove temp directory if empty
    if os.path.exists('temp_plots') and not os.listdir('temp_plots'):
        os.rmdir('temp_plots')
    
    if verbose:
        print(f"Bivariate analysis completed! Results saved to {filename}.xlsx")
    
    # Return None instead of the oot_binning_tables to avoid printing
    return None
def plot_single_dataset(
    ax1, df, bin_col_name, metric_column, y_axis_label, 
    totals_row, dataset_name, line_color='darkgoldenrod', bar_color='#0a3f7d',
    show_bar_values=False
) -> None:
    """
    Helper function to plot a single dataset (either training or OOT)
    """
    # Ensure bin column is string type
    df[bin_col_name] = df[bin_col_name].astype(str)
    
    # Identify regular, special, and missing bins
    regular_bins = df[~df[bin_col_name].str.contains('Special|Missing', regex=True, na=False)]
    special_bin = df[df[bin_col_name].str.contains('Special', regex=False, na=False)]
    missing_bin = df[df[bin_col_name].str.contains('Missing', regex=False, na=False)]
    
    # Create indices for x-axis
    x_indices = np.arange(len(df))
    
    # Format bin labels
    bin_labels = []
    for _, row in df.iterrows():
        bin_name = row[bin_col_name]
        if 'Special' in bin_name or 'Missing' in bin_name:
            bin_labels.append(bin_name)
        else:
            shortened_name = str(bin_name)
            if len(shortened_name) > 15:
                shortened_name = shortened_name[:12] + "..."
            bin_labels.append(shortened_name)
    
    # Plot Count (%) as bars
    bars = ax1.bar(x_indices, df['Count (%)'] * 100, color=bar_color, alpha=0.7)
    
    # Add value labels on top of bars if requested
    if show_bar_values:
        for i, bar in enumerate(bars):
            height = bar.get_height()
            ax1.text(
                bar.get_x() + bar.get_width()/2.,
                height + 0.5,
                f'{height:.1f}%',
                ha='center', va='bottom',
                fontsize=8, color='black'
            )
    
    ax1.set_xticks(x_indices)
    ax1.set_xticklabels(bin_labels, rotation=45, ha='right', fontsize=9)
    ax1.set_xlabel('Bins', fontsize=10)
    ax1.set_ylabel('Count (%)', color='blue', fontsize=10)
    ax1.tick_params(axis='y', labelcolor='blue', labelsize=9)
    
    # Create second y-axis for Event Rate or WoE
    ax2 = ax1.twinx()
    
    # Plot the metric line for regular bins only
    if not regular_bins.empty:
        regular_mask = ~df[bin_col_name].str.contains('Special|Missing', regex=True, na=False)
        regular_indices = np.where(regular_mask)[0]
        
        if len(regular_indices) > 0:
            ax2.plot(regular_indices, regular_bins[metric_column], 
                     marker='o', color=line_color, linewidth=2, label=f'{dataset_name} {y_axis_label}')
            
            # Add value annotations
            for idx, val in zip(regular_indices, regular_bins[metric_column]):
                ax2.annotate(f'{val:.3f}', 
                             xy=(idx, val), 
                             xytext=(0, 5),
                             textcoords='offset points',
                             ha='center', 
                             fontsize=7)
    
    # Plot Special bin point (if exists)
    if not special_bin.empty:
        special_indices = df[df[bin_col_name].str.contains('Special', regex=False, na=False)].index
        for idx in special_indices:
            special_val = df.loc[idx, metric_column]
            ax2.plot(idx, special_val,
                     marker='s', color='red', markersize=8, linestyle='None', 
                     label='Special' if idx == special_indices[0] else "")
            ax2.annotate(f'{special_val:.3f}', 
                         xy=(idx, special_val), 
                         xytext=(0, 5),
                         textcoords='offset points',
                         ha='center', 
                         fontsize=7)
    
    # Plot Missing bin point (if exists)
    if not missing_bin.empty:
        missing_indices = df[df[bin_col_name].str.contains('Missing', regex=False, na=False)].index
        for idx in missing_indices:
            missing_val = df.loc[idx, metric_column]
            ax2.plot(idx, missing_val,
                     marker='D', color='purple', markersize=8, linestyle='None', 
                     label='Missing' if idx == missing_indices[0] else "")
            ax2.annotate(f'{missing_val:.3f}', 
                         xy=(idx, missing_val), 
                         xytext=(0, 5),
                         textcoords='offset points',
                         ha='center', 
                         fontsize=7)
    
    # Add horizontal line for Totals if available
    if totals_row is not None:
        total_metric_value = totals_row[metric_column]
        
        # Handle case where value might be a blank string
        if isinstance(total_metric_value, str) and total_metric_value.strip() == '':
            total_metric_value = 0.0
        else:
            try:
                total_metric_value = float(total_metric_value)
            except (ValueError, TypeError):
                total_metric_value = 0.0
        
        ax2.axhline(y=total_metric_value, color=line_color, linestyle='--', 
                   alpha=0.7, label=f'{dataset_name} Total: {total_metric_value:.4f}')
        
        # Add IV information to legend if available
        if 'IV' in totals_row and totals_row['IV'] not in (None, '', np.nan):
            try:
                iv_value = float(totals_row['IV'])
                ax2.plot([], [], ' ', label=f'IV: {iv_value:.4f}')
            except (ValueError, TypeError):
                pass
    
    # Set y-axis label
    ax2.set_ylabel(y_axis_label, color=line_color, fontsize=11)
    ax2.tick_params(axis='y', labelcolor=line_color)
    
    # Add legend with unique entries
    handles, labels = ax2.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    ax2.legend(by_label.values(), by_label.keys(), loc='best', fontsize=9)


def plot_comparison(
    ax1, train_df, oot_df, bin_col_name, metric_column, 
    y_axis_label, train_totals_row, total_event_rate_oot, metric,
    show_bar_values=False
) -> None:
    """
    Helper function to plot a comparison between training and OOT datasets
    
    Parameters:
    -----------
    ax1 : matplotlib axis
        The primary axis for plotting
    train_df : pandas DataFrame
        Training data binning table
    oot_df : pandas DataFrame
        OOT data binning table
    bin_col_name : str
        Name of the bin column
    metric_column : str
        Name of the metric column ('Event rate' or 'WoE')
    y_axis_label : str
        Label for the y-axis
    train_totals_row : pandas Series
        Totals row from training data
    total_event_rate_oot : float
        Overall event rate for OOT data
    metric : str
        Metric to plot ('event_rate' or 'woe')
    show_bar_values : bool
        Whether to show Count (%) values on top of the bars
    """
    # Ensure bin columns are string type for consistency
    train_df[bin_col_name] = train_df[bin_col_name].astype(str)
    oot_df[bin_col_name] = oot_df[bin_col_name].astype(str)
    
    # Create a combined set of unique bin names, preserving order from train_df first
    bin_names = list(train_df[bin_col_name])
    # Add any bins from OOT that aren't in training (shouldn't happen normally)
    for bin_name in oot_df[bin_col_name]:
        if bin_name not in bin_names:
            bin_names.append(bin_name)
    
    # Create x indices
    x_indices = np.arange(len(bin_names))
    
    # Prepare data for plotting - match bin names between datasets
    train_values = []
    oot_values = []
    train_counts = []
    oot_counts = []
    
    for bin_name in bin_names:
        # Find metric values for each bin in each dataset
        train_row = train_df[train_df[bin_col_name] == bin_name]
        oot_row = oot_df[oot_df[bin_col_name] == bin_name]
        
        # If bin exists in training data, get its value; otherwise NaN
        if not train_row.empty:
            train_values.append(train_row[metric_column].values[0])
            train_counts.append(train_row['Count (%)'].values[0] * 100)  # Convert to percentage
        else:
            train_values.append(np.nan)
            train_counts.append(0)
        
        # If bin exists in OOT data, get its value; otherwise NaN
        if not oot_row.empty:
            oot_values.append(oot_row[metric_column].values[0])
            oot_counts.append(oot_row['Count (%)'].values[0] * 100)  # Convert to percentage
        else:
            oot_values.append(np.nan)
            oot_counts.append(0)
    
    # Format bin labels
    bin_labels = []
    for bin_name in bin_names:
        if 'Special' in bin_name or 'Missing' in bin_name:
            bin_labels.append(bin_name)
        else:
            shortened_name = str(bin_name)
            if len(shortened_name) > 15:
                shortened_name = shortened_name[:12] + "..."
            bin_labels.append(shortened_name)
    
    # Plot Count (%) as grouped bars
    bar_width = 0.35
    bar1 = ax1.bar(x_indices - bar_width/2, train_counts, bar_width, color='#0a3f7d', alpha=0.7, label='Train Count (%)')
    bar2 = ax1.bar(x_indices + bar_width/2, oot_counts, bar_width, color='#d35400', alpha=0.7, label='OOT Count (%)')
    
    # Add value labels on top of bars if requested
    if show_bar_values:
        for i, (b1, b2) in enumerate(zip(bar1, bar2)):
            # Training bar values
            h1 = b1.get_height()
            if h1 > 0:
                ax1.text(
                    b1.get_x() + b1.get_width()/2.,
                    h1 + 0.5,
                    f'{h1:.1f}%',
                    ha='center', va='bottom',
                    fontsize=8, color='navy'
                )
            
            # OOT bar values
            h2 = b2.get_height()
            if h2 > 0:
                ax1.text(
                    b2.get_x() + b2.get_width()/2.,
                    h2 + 0.5,
                    f'{h2:.1f}%',
                    ha='center', va='bottom',
                    fontsize=8, color='darkred'
                )
    
    ax1.set_xticks(x_indices)
    ax1.set_xticklabels(bin_labels, rotation=45, ha='right', fontsize=9)
    ax1.set_xlabel('Bins', fontsize=10)
    ax1.set_ylabel('Count (%)', color='blue', fontsize=10)
    ax1.tick_params(axis='y', labelcolor='blue', labelsize=9)
    ax1.legend(loc='upper left', fontsize=8)
    
    # Create second y-axis for Event Rate or WoE
    ax2 = ax1.twinx()
    
    # Identify regular, special, and missing bins
    regular_mask = ~np.array([('Special' in b or 'Missing' in b) for b in bin_names])
    regular_indices = np.where(regular_mask)[0]
    special_indices = [i for i, b in enumerate(bin_names) if 'Special' in b]
    missing_indices = [i for i, b in enumerate(bin_names) if 'Missing' in b]
    
    # Plot lines for regular bins
    if len(regular_indices) > 0:
        # Filter out NaN values for line plots
        valid_train_idx = [i for i in regular_indices if not np.isnan(train_values[i])]
        valid_oot_idx = [i for i in regular_indices if not np.isnan(oot_values[i])]
        
        if valid_train_idx:
            # Plot training data line
            train_line = ax2.plot([i for i in valid_train_idx], 
                                  [train_values[i] for i in valid_train_idx], 
                                  marker='o', color='darkgoldenrod', linewidth=2, 
                                  label=f'Train {y_axis_label}')
            
            # Add value annotations for training line
            for idx in valid_train_idx:
                ax2.annotate(f'{train_values[idx]:.3f}', 
                             xy=(idx, train_values[idx]), 
                             xytext=(0, 5),
                             textcoords='offset points',
                             ha='center', 
                             fontsize=7,
                             color='darkgoldenrod')
        
        if valid_oot_idx:
            # Plot OOT data line
            oot_line = ax2.plot([i for i in valid_oot_idx], 
                                [oot_values[i] for i in valid_oot_idx], 
                                marker='s', color='darkgreen', linewidth=2, 
                                label=f'OOT {y_axis_label}')
            
            # Add value annotations for OOT line
            for idx in valid_oot_idx:
                ax2.annotate(f'{oot_values[idx]:.3f}', 
                             xy=(idx, oot_values[idx]), 
                             xytext=(0, -15),
                             textcoords='offset points',
                             ha='center', 
                             fontsize=7,
                             color='darkgreen')
    
    # Plot Special bin points
    for idx in special_indices:
        if not np.isnan(train_values[idx]):
            ax2.plot(idx, train_values[idx],
                     marker='s', color='red', markersize=8, linestyle='None', 
                     label='Train Special' if idx == special_indices[0] else "")
            ax2.annotate(f'{train_values[idx]:.3f}', 
                         xy=(idx, train_values[idx]), 
                         xytext=(-10, 5),
                         textcoords='offset points',
                         ha='center', 
                         fontsize=7,
                         color='red')
        
        if not np.isnan(oot_values[idx]):
            ax2.plot(idx, oot_values[idx],
                     marker='s', color='darkred', markersize=8, linestyle='None', 
                     label='OOT Special' if idx == special_indices[0] else "")
            ax2.annotate(f'{oot_values[idx]:.3f}', 
                         xy=(idx, oot_values[idx]), 
                         xytext=(10, 5),
                         textcoords='offset points',
                         ha='center', 
                         fontsize=7,
                         color='darkred')
    
    # Plot Missing bin points
    for idx in missing_indices:
        if not np.isnan(train_values[idx]):
            ax2.plot(idx, train_values[idx],
                     marker='D', color='purple', markersize=8, linestyle='None', 
                     label='Train Missing' if idx == missing_indices[0] else "")
            ax2.annotate(f'{train_values[idx]:.3f}', 
                         xy=(idx, train_values[idx]), 
                         xytext=(-10, 5),
                         textcoords='offset points',
                         ha='center', 
                         fontsize=7,
                         color='purple')
        
        if not np.isnan(oot_values[idx]):
            ax2.plot(idx, oot_values[idx],
                     marker='D', color='darkmagenta', markersize=8, linestyle='None', 
                     label='OOT Missing' if idx == missing_indices[0] else "")
            ax2.annotate(f'{oot_values[idx]:.3f}', 
                         xy=(idx, oot_values[idx]), 
                         xytext=(10, 5),
                         textcoords='offset points',
                         ha='center', 
                         fontsize=7,
                         color='darkmagenta')
    
    # Add horizontal lines for totals
    if train_totals_row is not None and metric_column in train_totals_row:
        try:
            train_total_metric = float(train_totals_row[metric_column]) if train_totals_row[metric_column] != '' else 0.0
            ax2.axhline(y=train_total_metric, color='darkgoldenrod', linestyle='--', 
                       alpha=0.7, label=f'Train Total: {train_total_metric:.4f}')
        except (ValueError, TypeError):
            pass
    
    if total_event_rate_oot is not None and metric == 'event_rate':
        try:
            oot_total_metric = float(total_event_rate_oot)
            ax2.axhline(y=oot_total_metric, color='darkgreen', linestyle='--', 
                       alpha=0.7, label=f'OOT Total: {oot_total_metric:.4f}')
        except (ValueError, TypeError):
            pass
    
    # Add IV information to legend if available for training data
    if train_totals_row is not None and 'IV' in train_totals_row and train_totals_row['IV'] not in (None, '', np.nan):
        try:
            iv_value_train = float(train_totals_row['IV'])
            ax2.plot([], [], ' ', label=f'Train IV: {iv_value_train:.4f}')
        except (ValueError, TypeError):
            pass
    
    # Set y-axis label and legend
    ax2.set_ylabel(y_axis_label, color='black', fontsize=11)
    ax2.tick_params(axis='y', labelcolor='black')
    
    # Add legend with unique entries
    handles, labels = ax2.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    ax2.legend(by_label.values(), by_label.keys(), loc='upper right', fontsize=8)

In [ ]:
# Comparing training data with a newer time period
unified_bivariate_analysis(
    binning_process=binning_process,
    filename='train_validation_new7',
    metric='event_rate',
    oot_data=test_data,  # Data from a more recent time period
    target_column=target,
    compare_data=True ,
    show_bar_values=True # Show both datasets side by side
)

In [ ]:
# Get list of final selected variables
final_predictors=list(binning_process.get_support(names=True))
#final_predictors = ['loyalty_score','credit_score','customer_group']

In [ ]:
# Transform the data into woe or binned values or event rate
X_train_WOE = binning_process.transform(X_train,metric='woe')

In [ ]:
# Test example to understand the how binning object trasform the data to woe values
test_exmaple = pd.DataFrame([{'age': -999,
 'income': -0.8276048978295478,
 'experience_years': -0.7913895315832409,
 'credit_score': -0.7389923463082454,
 'avg_spend': 0.7331633070112105,
 'loyalty_score': -0.8702525202009143,
 'satisfaction_rating': 2.237864030340072,
 'demographic_index': -0.3085181724416579,
 'financial_status': -0.0191376466062278,
 'work_experience': -1.812865565832083,
 'credit_index': 0.5505331954414358,
 'customer_group': 'group_B'}])

test_WOE = binning_process.transform(test_exmaple,metric='woe')
test_WOE
